# 必要なライブラリをインポート

In [1]:
# サンプルデータセットが用意されているライブラリ sklearn.datasets から
# ボストンの住宅価格データを取得するためのメソッド load_boston をインポート
from sklearn.datasets import load_boston

# pandas のインポート
import pandas as pd

# 機械学習用ライブラリ sklearn（scikit-learn）内にあるライブラリ  から
# モデル構築（訓練用）/検証データ分割用メソッド train_test_split をインポート
from sklearn.model_selection import train_test_split

# 機械学習用ライブラリ sklearn（scikit-learn）から線形回帰用クラス linear_model をインポート 
from sklearn import linear_model

# 機械学習用ライブラリ sklearn（scikit-learn）内にあるライブラリ preprocessing から
# 標準化用クラス StandardScaler をインポート 
from sklearn.preprocessing import StandardScaler

# 統計解析用ライブラリ statsmodels 内にあるライブラリ stats.outliers_influence から
# 分散拡大係数（VIF）計算用メソッド variance_inflation_factor をインポート
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 精度評価指標を計算するためのメソッドをインポート
#   ・r2_score：決定係数
#   ・mean_squared_error：平均二乗誤差
#   ・mean_absolute_error：平均絶対誤差
#   ・median_absolute_error：Median Absolute Error
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, median_absolute_error

# オブジェクトのコピーを行うためのモジュール copy をインポート
import copy

# 数学的な関数を使うためのライブラリ math をインポート
import math

# グラフ描画用ライブラリ matplotlib、seaborn をインポート
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 分散拡大係数（VIF）を確認するための関数を定義しておく

In [2]:
# 投入したデータセットの全ての変数についてVIFを計算する関数 checkVIF の定義
def checkVIF( ExplanatoryVarDataSet ):
  tmp_columnList = ExplanatoryVarDataSet.columns
  vifList = []
  for i in range(len(tmp_columnList)):
    colname = tmp_columnList[i]
    vif = variance_inflation_factor(ExplanatoryVarDataSet.values, i)
    vifList.append( [ colname, vif ] )
  return  pd.DataFrame( vifList, columns=["COLUMN","VIF"] )

# データの読み込み

In [3]:
# ボストン住宅価格データを読み込む
loadBoston = load_boston()
boston = pd.DataFrame(loadBoston.data, columns = loadBoston.feature_names)
boston["MEDV"] = loadBoston.target
boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


# 全データをモデル構築用データ、モデル検証用データに分割する

In [4]:
# 目的変数と説明変数に分割
columnList = boston.columns.values.tolist()
columnList.remove("MEDV")
X = boston.loc[:,columnList]
y = boston.loc[:,["MEDV"]]

# モデル構築用データ、モデル検証用データに分割（70:30に分割）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(354, 13)
(152, 13)
(354, 1)
(152, 1)


# 標準化の実施

In [5]:
# X_train のデータを使い標準化パラメータを獲得してから、X_train、X_test に対して標準化を実施
scaler_X = StandardScaler()
scaler_X.fit( X_train )
X_train_std = pd.DataFrame(scaler_X.transform(X_train), columns=columnList)
X_test_std = pd.DataFrame(scaler_X.transform(X_test), columns=columnList)

In [6]:
# y_train のデータを使い標準化パラメータを獲得してから、y_train、y_test に対して標準化を実施
scaler_y = StandardScaler()
scaler_y.fit( y_train )
y_train_std = pd.DataFrame(scaler_y.transform(y_train), columns=["MEDV"])
y_test_std = pd.DataFrame(scaler_y.transform(y_test), columns=["MEDV"])

# 重回帰分析を実行する

In [19]:
# 重回帰分析を実行するためのインスタンスを生成
reg = linear_model.LinearRegression()

# 重回帰分析を実行（後で多重共線性が確認された場合（VIFが10以上）、columnList から変数を取り除く
# ※ columnList.remove("変数名") で削除可能）
reg.fit(X_train_std[columnList], y_train_std)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [20]:
# 分析結果として、回帰係数（reg.coef_）、切片（reg.intercept_）を表示する
print(columnList)
print(reg.coef_[0])
print(reg.intercept_[0])

['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']
[-0.1182196   0.0862307   0.0365998   0.08680927 -0.19178481  0.31080108
 -0.03188577 -0.31385735  0.22337309 -0.15434866 -0.21890703  0.10919653
 -0.41447035]
-1.3926342469197362e-16


In [21]:
# 多重共線性を確認する（）
vif = checkVIF(X_train_std[columnList])
vif["COEF"] = reg.coef_[0]
vif

,COLUMN,VIF,COEF
0,CRIM,1.731586,-0.118220
1,ZN,2.394452,0.086231
2,INDUS,3.836487,0.036600
3,CHAS,1.102642,0.086809
4,NOX,4.627219,-0.191785
5,RM,1.913056,0.310801
6,AGE,2.996110,-0.031886
7,DIS,3.971023,-0.313857
8,RAD,7.534681,0.223373
9,TAX,8.879471,-0.154349


In [22]:
# モデル構築用データについて各精度評価指標を計算する
print("R2 SCORE:" + str(r2_score(y_train_std, reg.predict(X_train_std[columnList]))) )
print("MSE:" + str(mean_squared_error(y_train_std, reg.predict(X_train_std[columnList]))) )
print("MAE:" + str(mean_absolute_error(y_train_std, reg.predict(X_train_std[columnList]))) )
print("MedianAE:" + str(median_absolute_error(y_train_std, reg.predict(X_train_std[columnList]))) )

R2 SCORE:0.7434997532004697
MSE:0.25650024679953026
MAE:0.35804949691064236
MedianAE:0.2703717627040221


In [23]:
# モデル検証用データについて各精度評価指標を計算する
print("R2 SCORE:" + str(r2_score(y_test_std, reg.predict(X_test_std[columnList]))) )
print("MSE:" + str(mean_squared_error(y_test_std, reg.predict(X_test_std[columnList]))) )
print("MAE:" + str(mean_absolute_error(y_test_std, reg.predict(X_test_std[columnList]))) )
print("MedianAE:" + str(median_absolute_error(y_test_std, reg.predict(X_test_std[columnList]))) )

R2 SCORE:0.7112260057484933
MSE:0.2448042530771131
MAE:0.3373443885651479
MedianAE:0.2530256207982074
